In [2]:
import cv2
import numpy as np

# 读取输入图像
img = cv2.imread('D:/Game/upian/uling.png')

# 将图像转为灰度图
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# 对灰度图进行中值滤波降噪
gray = cv2.medianBlur(gray,5)

# 使用自适应阈值法二值化图像
ret,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# 对二值化后的图像进行形态学腐蚀操作，去除噪点
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,iterations=2)

# 对形态学腐蚀后的图像进行形态学膨胀操作，填充图像内部空洞
sure_bg = cv2.dilate(opening,kernel,iterations=3)

# 寻找图像中前景物体的轮廓，得到mask
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

# 对mask进行标记，将无法确定前景或背景的区域标记为0
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown==255] = 0

# 使用分水岭算法，分割出图像中的前景物体
markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]

# 显示结果
cv2.imshow('input',img)
cv2.waitKey(0)
cv2.destroyAllWindows()